# Movie Recommender System with Quantum Computing

This notebook implements a movie recommender system using quantum computing and topological data analysis.

In [ ]:
# Install dependencies
!pip install -q torch-geometric==2.3.1 qiskit==1.0.2 qiskit-machine-learning==0.5.0 qiskit-aer==0.12.0 pennylane==0.30.0 fastdtw==0.3.4
!pip install -q gudhi==3.7.0 pot==0.9.0

In [ ]:
import os
import sys
import logging
from pathlib import Path
import torch
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from gudhi import RipsComplex
import ot
from scipy.stats import hypergeom
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
import pennylane as qml
from pennylane import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
# Create necessary directories
os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

In [ ]:
# Download MovieLens 100K dataset
!wget -q https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip
!mv ml-100k/* data/
!rm -rf ml-100k ml-100k.zip

In [ ]:
# Copy project files to Kaggle environment
!mkdir -p src/data src/models src/analysis
!cp -r /kaggle/input/movie-recommender-project/* .

In [ ]:
# Import project modules
from src.data.processor import DataProcessor
from src.models.quantum_model import QuantumModel
from src.analysis.analyzer import TopologicalAnalyzer, OutlierDetector, FairnessAnalyzer

In [ ]:
# Load and preprocess data
logger.info("Loading and preprocessing data...")
data_processor = DataProcessor()
data_processor.load_data()
data_processor.preprocess_data()
ratings, users, movies, graph, edge_index, edge_attr, fft_values = data_processor.get_processed_data()
data_processor.save_processed_data(Path('results/processed_data'))

In [ ]:
# Perform topological analysis
logger.info("Performing topological analysis...")
topological_analyzer = TopologicalAnalyzer()
persistence, landscape = topological_analyzer.analyze_genres(movies)
topological_analyzer.save_analysis(Path('results/topology'))

In [ ]:
# Detect outliers
logger.info("Detecting outliers...")
outlier_detector = OutlierDetector()
outlier_scores = outlier_detector.detect_outliers(ratings, users)
outlier_detector.save_outliers(Path('results/outliers'))

In [ ]:
# Analyze fairness
logger.info("Analyzing fairness...")
fairness_analyzer = FairnessAnalyzer()
fairness_scores = fairness_analyzer.analyze_fairness(ratings, users)
fairness_analyzer.save_fairness(Path('results/fairness'))

In [ ]:
# Prepare data for quantum model
logger.info("Preparing data for quantum model...")
X = torch.tensor(ratings[['rating', 'time_decay']].values, dtype=torch.float32)
y = torch.tensor(ratings['rating'].values, dtype=torch.float32).view(-1, 1)

In [ ]:
# Initialize and train quantum model
logger.info("Training quantum model...")
model = QuantumModel(n_qubits=2, n_layers=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)

n_epochs = 10
batch_size = 32
n_batches = len(X) // batch_size

for epoch in tqdm(range(n_epochs), desc="Training"):
    epoch_loss = 0.0
    for batch_idx in range(n_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        
        X_batch = X[start_idx:end_idx]
        y_batch = y[start_idx:end_idx]
        
        loss = model.train_step(X_batch, y_batch, optimizer)
        epoch_loss += loss
    
    avg_loss = epoch_loss / n_batches
    logger.info(f"Epoch {epoch + 1}/{n_epochs}, Loss: {avg_loss:.4f}")

In [ ]:
# Save trained model
logger.info("Saving trained model...")
model.save(Path('models/quantum_model.pt'))

In [ ]:
# Make predictions
logger.info("Making predictions...")
predictions = model.predict(X)

# Save predictions
pd.DataFrame({
    'user_id': ratings['user_id'],
    'movie_id': ratings['movie_id'],
    'actual_rating': ratings['rating'],
    'predicted_rating': predictions.numpy().flatten()
}).to_csv('results/predictions.csv', index=False)

In [ ]:
# Visualize results
plt.figure(figsize=(12, 6))
sns.scatterplot(x='actual_rating', y='predicted_rating', data=pd.read_csv('results/predictions.csv'))
plt.title('Actual vs Predicted Ratings')
plt.xlabel('Actual Rating')
plt.ylabel('Predicted Rating')
plt.savefig('results/predictions_plot.png')
plt.close()